## Introduction

A widely stated (and disputed) result is the "10 days" rule — that if an investor were to exclude the best 10 days in market history then the stock market would be a horrible investment, and conversely, excluding the worst 10 days in market history would make it fantastic. Seyhun [1] visualized this result with various levels of outlier exclusion: 

<img src="imgs/Seyhun-viz.png">

Regardless of the degree to which such a result is overstated or relies on faulty assumptions, it is clear that evaluating tail end risks (and rewards) are crucial to understanding the success of financial strategies and institutions. Such "Black Swan" or "Tail End" events may be responsible for the vast majority of returns and losses on a given strategy. 

In this notebook, I hope to illustrate some examples of how tail end risk can affect investment strategies the way they're traditionally understood. I hope to illustrate some alternatives as well.

## Correlation Shmorrelation

Assume that what funds do is as follows: develop a signal that has above $\rho_{threshold}$ correlation on historical back-tested data, and then trade in proportion to the strength of that signal. 

As a few simplifying assumptions, let: 
1. The signal $X$ is normally distributed with 0 mean and standard deviation 1: $X \sim N(0, 1)$
2. The daily return $Y$ is additive and also normally distributed with 0 mean and standard deviation 1: $Y \sim N(0, 1)$
3. $X$ and $Y$ have a correlation of $\rho$, that is: $\frac{E[(X - \mu_{X})(Y - \mu_{Y})]}{\sigma_{X} \sigma_{Y}} = E[X Y] = 1$

**(Note that this normality assumption often used in financial models may also itself be violated by certain tail risk challenges, but we will ignore this for now.)**

We wish to assess the conditional probability that $Y > K$ given that $X > K$, that is:

$P(Y > K | X > K) = \frac{P(X > K, Y > K)}{P(X > K)}$

Another interpretation of this expression is that this is the ratio of the probability of both $X$ and $Y$ exceeding some threshold $K$ assuming correlation $\rho$ over the probability of both $X$ and $Y$ exceeding $K$ assuming correlation $1$ (given that if the correlation is 1, then $P(X > K, Y > K) = P(X > K)$.

The evaluation of this expression is performed as follows: 

In [4]:
import numpy as np

np.random.seed(42)

mu = 0
sigma = 1
X = np.random.normal(mu, sigma, 1000)

R = [1 0.75; 0.75 1];

"""
R = [1 0.75; 0.75 1];
L = chol(R)
M = M*L;
x = M(:,1);
y = M(:,2);
corr(x,y)
"""

[ 4.96714153e-01 -1.38264301e-01  6.47688538e-01  1.52302986e+00
 -2.34153375e-01 -2.34136957e-01  1.57921282e+00  7.67434729e-01
 -4.69474386e-01  5.42560044e-01 -4.63417693e-01 -4.65729754e-01
  2.41962272e-01 -1.91328024e+00 -1.72491783e+00 -5.62287529e-01
 -1.01283112e+00  3.14247333e-01 -9.08024076e-01 -1.41230370e+00
  1.46564877e+00 -2.25776300e-01  6.75282047e-02 -1.42474819e+00
 -5.44382725e-01  1.10922590e-01 -1.15099358e+00  3.75698018e-01
 -6.00638690e-01 -2.91693750e-01 -6.01706612e-01  1.85227818e+00
 -1.34972247e-02 -1.05771093e+00  8.22544912e-01 -1.22084365e+00
  2.08863595e-01 -1.95967012e+00 -1.32818605e+00  1.96861236e-01
  7.38466580e-01  1.71368281e-01 -1.15648282e-01 -3.01103696e-01
 -1.47852199e+00 -7.19844208e-01 -4.60638771e-01  1.05712223e+00
  3.43618290e-01 -1.76304016e+00  3.24083969e-01 -3.85082280e-01
 -6.76922000e-01  6.11676289e-01  1.03099952e+00  9.31280119e-01
 -8.39217523e-01 -3.09212376e-01  3.31263431e-01  9.75545127e-01
 -4.79174238e-01 -1.85658

'\nR = [1 0.75; 0.75 1];\nL = chol(R)\nM = M*L;\nx = M(:,1);\ny = M(:,2);\ncorr(x,y)\n'

## Hedge Fund Incentive Structures

## References

1. STOCK MARKET EXTREMES AND PORTFOLIO PERFORMANCE. A study commissioned by Towneley Capital Management and conducted by Professor H. Nejat Seyhun, University of Michigan. 1993. http://www.towneley.com/wp-content/uploads/2016/01/Research.-TCM-Mkt-Timing-Study-1993.pdf
2. Nassim Taleb's Twitter. https://twitter.com/nntaleb/status/1135116646442590208. 2019
3. "Why did the Crisis of 2008 Happen?" Nassim Taleb. 2010. 